In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from keras.datasets import mnist
import tensorflow.keras as kb
from tensorflow.keras import backend
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer


from plotnine import *

from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score

from sklearn.linear_model import LinearRegression # Linear Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv
from sklearn.model_selection import LeaveOneOut #LOO cv


# Together
Neural Networks are more of a framework for building custom models than a specific type of model. When building a neural network you need to think about 3 main things:

1. The structure of your model (how many layers? how many nodes? what activation functions?)
2. The loss function for your model
3. The optimizer you'll use to train (we'll get to that in the next NN lecture)

Your input and output layers are pre-determined by the number and type of input you want to go into your model, and the form of output that you want. For example for a regression problem you likely want a single node in the output layer. For a multi-class classification problem, you want `n` nodes in the output layer, one for each possible output category. 


## Review

### Layers, Hidden Layers, Nodes
<img src="https://drive.google.com/uc?export=view&id=1iN4PCbZyIDQCgKx9odEpTaXpHOc-vZzg" alt="Q" width = "400"/>

### Calculating the Value of a Node
<img src="https://drive.google.com/uc?export=view&id=1P_jfOCXs4wFUVpyFeRH3ysAIaeoHSe54" alt="Q" width = "400"/>

### Activation Functions
<img src="https://drive.google.com/uc?export=view&id=1CXSZZ9NjMlvdtopwGD5mAlFMukAmPBrQ" alt="Q" width = "400"/>
<img src="https://drive.google.com/uc?export=view&id=1I9vs6IgnHlohBkTt0G7E-vHLBQTT169_" alt="Q" width = "400"/>

Softmax:
$$ f(x) = \frac{e^{z_i}}{\sum_{j=1}^N e^z_j}$$

The softmax activation creates a vector of probabilities that sum to 1, and as a bonus, it's ✨differentiable✨.



In [2]:
x = np.array([1,6,7,8,3,2])
soft_x = np.exp(x)
soft_x = soft_x/np.sum(soft_x)
print(np.sum(soft_x))
np.round(soft_x,3)

1.0


array([0.001, 0.089, 0.243, 0.661, 0.004, 0.002])



## Building a Simple NN


In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/Music_data.csv")
feats = ["danceability", "energy", "loudness","acousticness", "instrumentalness", "liveness", "duration_ms"]
predict = "valence"

df.head()


,Unnamed: 0,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,duration_ms,track_name
0,1,Jonas Brothers,0.594,0.464,0,-7.898,1,0.0340,0.306000,0.000000,0.1870,0.545,193866,Please Be Mine
1,2,Jonas Brothers,0.582,0.697,7,-3.842,1,0.0695,0.000745,0.000000,0.1850,0.701,153346,S.O.S.
2,3,Jonas Brothers,0.442,0.961,1,-2.914,1,0.0928,0.000035,0.000036,0.0619,0.405,168480,Mandy
3,4,Jonas Brothers,0.659,0.857,11,-5.850,1,0.0437,0.004500,0.000002,0.3350,0.798,201960,Year 3000
4,5,Jonas Brothers,0.468,0.893,11,-3.554,1,0.1300,0.003890,0.000000,0.3670,0.810,164973,Hold On


In [9]:
# Regression

X = df[feats]
y = df["valence"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

z = StandardScaler()
X_train[feats] = z.fit_transform(X_train[feats])
X_test[feats] = z.transform(X_test[feats])

#structure of the model

model = kb.Sequential([
    kb.layers.Dense(1, input_shape = [7]),
])

#how to train the model

model.compile(loss = "mean_squared_error",
    optimizer = kb.optimizers.SGD())

#fit the model (same as SKlearn)

model.fit(X_train, y_train, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
64/64 [==============================] - 0s 3ms/step - loss: 1.1342 - val_loss: 0.2911
Epoch 2/100
64/64 [==============================] - 0s 2ms/step - loss: 0.1568 - val_loss: 0.0802
Epoch 3/100
64/64 [==============================] - 0s 3ms/step - loss: 0.0606 - val_loss: 0.0429
Epoch 4/100
64/64 [==============================] - 0s 3ms/step - loss: 0.0414 - val_loss: 0.0348
Epoch 5/100
64/64 [==============================] - 0s 2ms/step - loss: 0.0367 - val_loss: 0.0327
Epoch 6/100
64/64 [==============================] - 0s 2ms/step - loss: 0.0353 - val_loss: 0.0324
Epoch 7/100
64/64 [==============================] - 0s 2ms/step - loss: 0.0349 - val_loss: 0.0322
Epoch 8/100
64/64 [==============================] - 0s 2ms/step - loss: 0.0347 - val_loss: 0.0323
Epoch 9/100
64/64 [==============================] - 0s 2ms/step - loss: 0.0346 - val_loss: 0.0324
Epoch 10/100
64/64 [==============================] - 0s 2ms/step - loss: 0.0345 - val_loss: 0.0325
Epoch 11/

In [10]:
# get weights from Neural Network
model.get_weights()

[array([[ 0.1078383 ],
        [ 0.09139112],
        [-0.01775693],
        [ 0.02858892],
        [-0.00499878],
        [-0.00615598],
        [-0.03469113]], dtype=float32),
 array([0.4728315], dtype=float32)]

# In Class

## Building a Logistic Regression NN
Let's change the model we built together so that it predicts `mode` instead of valence. Because we're now predicting a categorical variable (or rather, we're predicting the probability that the data is in a given category), we need to change the last layer of our network. Copy and paste the model we made together, and let's make some changes. 

- Change your `y` to be `mode` 
- Add a `activation = "sigmoid"` argument to your Layer
- Change the `loss` argument in your `model.compile()` function to be `"binary_crossentropy"` (remember binary cross entropy is another name for log loss which we learned about in 392)


Then fit your model and see how well it does!



In [11]:
# change y and redo TTS

X = df[feats]
y = df["mode"]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

z = StandardScaler()
X_train[feats] = z.fit_transform(X_train[feats])
X_test[feats] = z.transform(X_test[feats])


In [25]:
#structure of the model

model = kb.Sequential([
    kb.layers.Dense(1, input_shape=[7], activation = 'sigmoid')
])

#how to train the model

model.compile(loss = "binary_crossentropy",
    optimizer = kb.optimizers.SGD())

#fit the model (same as SKlearn)
model.fit(X_train, y_train, epochs = 100, validation_data = (X_test, y_test))


Epoch 1/100
64/64 [==============================] - 1s 4ms/step - loss: 0.8931 - val_loss: 0.8127
Epoch 2/100
64/64 [==============================] - 0s 2ms/step - loss: 0.8355 - val_loss: 0.7739
Epoch 3/100
64/64 [==============================] - 0s 2ms/step - loss: 0.7901 - val_loss: 0.7433
Epoch 4/100
64/64 [==============================] - 0s 2ms/step - loss: 0.7544 - val_loss: 0.7190
Epoch 5/100
64/64 [==============================] - 0s 2ms/step - loss: 0.7261 - val_loss: 0.6995
Epoch 6/100
64/64 [==============================] - 0s 2ms/step - loss: 0.7040 - val_loss: 0.6839
Epoch 7/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6865 - val_loss: 0.6715
Epoch 8/100
64/64 [==============================] - 0s 1ms/step - loss: 0.6729 - val_loss: 0.6616
Epoch 9/100
64/64 [==============================] - 0s 1ms/step - loss: 0.6621 - val_loss: 0.6534
Epoch 10/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6537 - val_loss: 0.6468
Epoch 11/

## Making that Model DEEP

Now, let's modify your model once more, this time to add hidden layers.

- Your first `Dense()` layer should still have an `input_shape` of `[7]` but now you can make that hidden layer ANY SIZE you want
- add at LEAST 3 more `Dense()` layers with however many nodes you want (play around with it)
- make sure your last `Dense()` layer has an output dimension of `1` and a `sigmoid` activation
- compile and fit your model

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

How does the performance of your model change from the very simple NN you built before? Is there any overfitting going on?

In [26]:
### YOUR CODE HERE ###


#structure of the model

model = kb.Sequential([
    kb.layers.Dense(16, input_shape=[7], activation='sigmoid'),
    kb.layers.Dense(8, activation='sigmoid'),
    kb.layers.Dense(4, activation='sigmoid'),
    kb.layers.Dense(1, activation='sigmoid')
])

#how to train the model

model.compile(loss = "binary_crossentropy",
    optimizer = kb.optimizers.SGD())

#fit the model (same as SKlearn)
model.fit(X_train, y_train, epochs = 100, validation_data = (X_test, y_test))


Epoch 1/100
64/64 [==============================] - 1s 4ms/step - loss: 0.6835 - val_loss: 0.6629
Epoch 2/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6545 - val_loss: 0.6414
Epoch 3/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6403 - val_loss: 0.6302
Epoch 4/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6333 - val_loss: 0.6246
Epoch 5/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6299 - val_loss: 0.6215
Epoch 6/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6281 - val_loss: 0.6198
Epoch 7/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6272 - val_loss: 0.6187
Epoch 8/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6267 - val_loss: 0.6182
Epoch 9/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6264 - val_loss: 0.6177
Epoch 10/100
64/64 [==============================] - 0s 2ms/step - loss: 0.6263 - val_loss: 0.6175
Epoch 11/

- Use `.predict()` to grab the train/test set predictions from your model
- grab the ROC/AUC value

In [28]:
# grab predictions

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

# predict probabilities on train set

print(y_pred_train)

# assess performance

print("Train AUC: ", roc_auc_score(y_train, y_pred_train))
print("Test AUC: ", roc_auc_score(y_test, y_pred_test))



[[0.68168384]
 [0.68062186]
 [0.6819038 ]
 ...
 [0.6852464 ]
 [0.6873199 ]
 [0.68090796]]
Train AUC:  0.56442999267841
Test AUC:  0.547266904170715


In [19]:
# calculate ROC/AUC

auc_train = roc_auc_score(y_train, train_preds)
auc_test = roc_auc_score(y_test, test_preds)

print(train_preds)
print(test_preds)

[[0.68240446]
 [0.682346  ]
 [0.67949116]
 ...
 [0.6790647 ]
 [0.6782267 ]
 [0.6815529 ]]
[[0.6809473 ]
 [0.6819583 ]
 [0.68253535]
 [0.67950916]
 [0.6829001 ]
 [0.6798272 ]
 [0.680624  ]
 [0.6764436 ]
 [0.6785183 ]
 [0.6811077 ]
 [0.6826779 ]
 [0.6831749 ]
 [0.6833189 ]
 [0.67897457]
 [0.6833773 ]
 [0.67509484]
 [0.6822651 ]
 [0.68265986]
 [0.68148965]
 [0.6840098 ]
 [0.6815848 ]
 [0.6830555 ]
 [0.6746122 ]
 [0.6767001 ]
 [0.6828811 ]
 [0.67451906]
 [0.68112755]
 [0.6830908 ]
 [0.68423545]
 [0.67526966]
 [0.68295646]
 [0.68391424]
 [0.68301535]
 [0.6830708 ]
 [0.68324167]
 [0.67694855]
 [0.6835274 ]
 [0.68118715]
 [0.68224716]
 [0.68321896]
 [0.6825776 ]
 [0.6795639 ]
 [0.6841161 ]
 [0.6839787 ]
 [0.68109417]
 [0.68295205]
 [0.6802069 ]
 [0.68248963]
 [0.6792517 ]
 [0.68248546]
 [0.6828546 ]
 [0.68088233]
 [0.68325424]
 [0.6759792 ]
 [0.6837053 ]
 [0.6841244 ]
 [0.6806557 ]
 [0.68288094]
 [0.68189764]
 [0.6831915 ]
 [0.680529  ]
 [0.6793625 ]
 [0.6817854 ]
 [0.681037  ]
 [0.6794043 ]




## Building a Feed Forward NN for Black and White Digit Images

The famous dataset, MNIST, contains thousands of black and white pictures of handwritten digits. While other NN structures (like Convolutional NNs that we cover next) are built specifically to learn from images, when the images are simple and Black and White, even a Feed Forward NN can perform okay!

So let's build a NN that classifies handwritten digits.

<center><img src="https://drive.google.com/uc?export=view&id=1xZhgplIHkTkbKoL6SuWGHn_DL0AbNSPh" alt="mnist data" width = "500px" /></center>


In [29]:
# load and rescale the data so that the pixel values are between 0-1

# load data
((trainX, trainY), (testX, testY)) = mnist.load_data()

# reshape datta to be individual columns instead of image matrix
trainX = trainX.reshape((trainX.shape[0], 28 * 28 * 1))
testX = testX.reshape((testX.shape[0], 28 * 28 * 1))

# rescale data to be 0-1 instead of 0-255
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# change the labels to be in the correct format
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

11501568/11490434 [==============================] - 2s 0us/step


- Build a Deep Neural Network with at least 5 `Dense()` layers (but you can have 7, 10, or more!)
- the first layer needs an `input_shape` of `[784]` because there are 784 pixels per image (28x28)
- the last layer needs to have 10 nodes (one for each digit)
- the last layer needs to use a softmax activation so that each of our nodes represents the probability of being each digit
- you can choose the # of nodes for each of the hidden layers (typically we reduce the number of nodes each layer but that's not required)
- compile your model using `loss = "categorical_crossentropy"`, and either SGD or Adam as your optimizer
- choose a reasonable learning rate or use default
- during training, ask your model to print out the accuracy of the model
- train model for 100+ epochs

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

How does your model do? Are you surprised by the performance of the model given its structure isn't designed to handle images?

### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>

We fed in a `784x1` array/vector of pixel values (i.e. we had 784 features) into our Feed Forward NN. Does this structure take into acount the spatial nature of an image (where pixels surrounding a pixel are likely related)? Do you think that type of structure would be helpful for processing more complicated images?



In [34]:
# build structure of the model

model = kb.Sequential([
    kb.layers.Dense(18, input_shape=[784], activation='softmax'),
    kb.layers.Dense(17, activation='softmax'),
    kb.layers.Dense(16, activation='softmax'),
    kb.layers.Dense(15, activation='softmax'),
    kb.layers.Dense(14, activation = 'softmax'),
    kb.layers.Dense(13, activation = 'softmax'),
    kb.layers.Dense(10, activation = 'softmax'),

])

#how to train the model

model.compile(loss = "categorical_crossentropy",
    optimizer = kb.optimizers.SGD())

#fit the model (same as SKlearn)
model.fit(trainX, trainY, epochs = 100)

# compile model


#fit the model (same as SKlearn)

# predict probabilities on train set

print(y_pred_train)

# assess performance

print("Train AUC: ", roc_auc_score(y_train, y_pred_train))
print("Test AUC: ", roc_auc_score(y_test, y_pred_test))


Epoch 1/100
1875/1875 [==============================] - 3s 1ms/step - loss: 2.3027
Epoch 2/100
1875/1875 [==============================] - 2s 1ms/step - loss: 2.3013
Epoch 3/100
1390/1875 [=====================>........] - ETA: 0s - loss: 2.3011

KeyboardInterrupt: 

- Grab the weights from your model using `get_weights()`.


### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>


How many weights are there? Are these weights *interpretable* (e.g. with coefficients in a linear regression, the coefficients are interpretable...they easily tell us the relationship between our inputs and our outputs)? If I wanted to know the exact effect of the first feature/pixel on the predicted outcome, would you be able to grab that easily? Does the word black box seem appropriate?


### Question
<img src="https://drive.google.com/uc?export=view&id=1ghyQPx1N8dmU3MV4TrANvqNhGwnLni72" alt="Q" width = "200"/>


Using this image, identify which nodes' values are affected by the first pixel value (highlighted).

<center><img src="https://drive.google.com/uc?export=view&id=1XQh1nvMax1Hc5r9ITz-RiJDasyT5-V2b" alt="Q" width = "700"/></center>

